In [1]:
! pip3 install psycopg2-binary --user

import pandas as pd
import psycopg2
import numpy as np
from getpass import getpass

In [2]:
# connect to database
connection = psycopg2.connect(
    database  = "postgres",
    user      = "postgres",
    password  = getpass(),
    host      = "movie-rec-scrape.cvslmiksgnix.us-east-1.rds.amazonaws.com",
    port      = '5432'
)


··········


In [21]:
# cursor object
try:
    c = connection.cursor()
    print("Connected!")
except:
    print("Connection problem chief!")

Connected!


In [0]:
c.execute("SELECT movie_id, username FROM reviews WHERE user_rating > 6")

In [0]:
result = c.fetchall()
# result

In [24]:
df = pd.DataFrame(result, columns = ['movieid', 'userid'])
df.head()

,movieid,userid
0,3120280,mary-179-677383
1,3120280,mhpayne78
2,3120280,marymcnamara-69052
3,3120280,danae-gr-p
4,3120280,a_lowther


In [0]:
df.movieid = df.movieid.str.lstrip('0')

In [0]:
url = 'https://datasets.imdbws.com/title.basics.tsv.gz'

In [27]:
df2 = pd.read_table(url, sep='\t')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df2['movieid'] =[i[2:].lstrip('0') for i in df2['tconst']]

In [0]:
df2['movieid'] = df2['movieid']

In [30]:
df2.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,movieid
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",1
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",2
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",3
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,\N,"Animation,Short",4
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",5


In [0]:
df = df.merge(df2, how='inner', on='movieid')

In [0]:
df.

In [32]:
df.movieid.value_counts()

7286456    7512
111161     6797
4154796    6483
468569     5594
120737     4465
           ... 
1905085       1
138388        1
64853         1
251900        1
1345525       1
Name: movieid, Length: 104768, dtype: int64

In [0]:
movies = df.movieid.unique().tolist()

In [0]:
df['users'] = df['userid'].astype('str')

In [35]:
users = df.userid.unique().tolist()
len(users)

804614

In [36]:
df.dtypes

movieid           object
userid            object
tconst            object
titleType         object
primaryTitle      object
originalTitle     object
isAdult            int64
startYear         object
endYear           object
runtimeMinutes    object
genres            object
users             object
dtype: object

In [37]:
# ! sudo su
! yum update -y
! yum -y install python-pip
! python -V

/bin/bash: yum: command not found
/bin/bash: yum: command not found
Python 3.6.9


In [38]:
! which pip

/usr/local/bin/pip


In [39]:
! python -m pip install tqdm
# ! python -c 'import tqdm'
! python -m pip install gensim
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline

import warnings;
warnings.filterwarnings('ignore')



# shuffle usernames
random.shuffle(users)

# extract 90% of users
users_train = [users[i] for i in range(round(0.9*len(users)))]

# split data into train and validation set
train_df = df[df['users'].isin(users_train)]
validation_df = df[~df['users'].isin(users_train)]

train_df.head()

,movieid,userid,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,users
1,3120280,mhpayne78,tt3120280,movie,Sierra Burgess Is a Loser,Sierra Burgess Is a Loser,0,2018,\N,105,"Comedy,Drama,Romance",mhpayne78
2,3120280,marymcnamara-69052,tt3120280,movie,Sierra Burgess Is a Loser,Sierra Burgess Is a Loser,0,2018,\N,105,"Comedy,Drama,Romance",marymcnamara-69052
3,3120280,danae-gr-p,tt3120280,movie,Sierra Burgess Is a Loser,Sierra Burgess Is a Loser,0,2018,\N,105,"Comedy,Drama,Romance",danae-gr-p
4,3120280,a_lowther,tt3120280,movie,Sierra Burgess Is a Loser,Sierra Burgess Is a Loser,0,2018,\N,105,"Comedy,Drama,Romance",a_lowther
5,3120280,schylar-73786,tt3120280,movie,Sierra Burgess Is a Loser,Sierra Burgess Is a Loser,0,2018,\N,105,"Comedy,Drama,Romance",schylar-73786


In [0]:
# enable multiprocessing
import multiprocessing.dummy as mp


In [0]:
# list to capture watch history of the users
watched_train = []

def history_juggler(i):
    temp = train_df[train_df["users"] == i]["movieid"].tolist()
    watched_train.append(temp)

# populate the list with the movie ids
if __name__=="__main__":
  p=mp.Pool(4)
  p.map(history_juggler, users_train) # removed tqdm since it doesn't play nice with mp
  p.close()
  p.join()

In [0]:
wsaa# list to capture purchase history of the customers
watched_val = []

# populate the list with the product codes
for i in tqdm(validation_df['users'].unique()):
    temp = validation_df[validation_df["users"] == i]["movieid"].tolist()
    watched_val.append(temp)

100%|██████████| 34641/34641 [04:53<00:00, 117.83it/s]


In [0]:
watched_train = [[str(x) for x in lst] for lst in watched_train]

In [0]:
watched_train[0]

['6110504']

In [0]:
watched_val = [[str(x) for x in lst] for lst in watched_val]

In [0]:
# train word2vec model
model = Word2Vec(window = 10, sg = 1, hs = 0,
                 negative = 10, # for negative sampling
                 alpha=0.03, min_alpha=0.0007,
                 seed = 14)

model.build_vocab(watched_train, progress_per=200)

model.train(watched_train, total_examples = model.corpus_count, 
            epochs=10, report_delay=1)

(8195290, 9259500)

In [0]:
# save word2vec model
model.save("word2vec_2.model")

In [0]:
import gensim

model = gensim.models.Word2Vec.load("word2vec_2.model")

In [0]:
model.init_sims(replace=True)

In [0]:
print(model)

In [0]:
# extract all vectors
X = model[model.wv.vocab]

X.shape

(49385, 100)

In [0]:
# ! sudo pip install umap-learn

In [0]:
train_df['movieid'] = train_df['movieid'].astype(str)
train_df['movieid'][0]

'72777'

In [0]:
movies = train_df[["movieid", "primaryTitle"]]

# remove duplicates
movies.drop_duplicates(inplace=True, subset='movieid', keep="last")

# create product-ID and product-description dictionary
movies_dict = movies.groupby('movieid')['primaryTitle'].apply(list).to_dict()

In [0]:
# test the dictionary
movies_dict['105112']



['Patriot Games']

In [0]:
def similar_movies(v, n = 6):
    
    # extract most similar movies for the input vector
    ms = model.similar_by_vector(v, topn= n+1)[1:]
    
    # extract title and similarity score of the similar movies
    new_ms = []
    for j in ms:
        pair = (movies_dict[j[0]][0], j[1])
        new_ms.append(pair)
        
    return new_ms

In [0]:
similar_movies(model['105112'])

[('How the Grinch Stole Christmas', 0.730616569519043),
 ('Split Second', 0.7187882661819458),
 ('The Final Countdown', 0.7176185846328735),
 ("Livin' Large!", 0.7173690795898438),
 ('Hawks', 0.7129511833190918),
 ('Manhunter', 0.7002811431884766)]

In [0]:
def aggregate_vectors(movies):
    movie_vec = []
    for i in movies:
        try:
            movie_vec.append(model[i])
        except KeyError:
            continue
        
    return np.mean(movie_vec, axis=0)


In [0]:
len(watched_val[2])

597

In [0]:
aggregate_vectors(watched_val[2]).shape

(100,)

In [0]:
similar_movies(aggregate_vectors(watched_val[2]))

[('Our Miss Fred', 0.9716984629631042),
 ('Lovefilm', 0.9665443301200867),
 ('Gene-X', 0.9657200574874878),
 ('O Pátio das Cantigas', 0.9629855751991272),
 ('Mercano the Martian', 0.9613674283027649),
 ("Hitler's Grave", 0.96126788854599)]

In [0]:
similar_movies(aggregate_vectors(watched_val[2][-10:]))

[('South Sea Woman', 0.9722521305084229),
 ('The Joyless Street', 0.969933807849884),
 ('Ladies in Retirement', 0.9696546792984009),
 ('The Singing Kid', 0.9678635597229004),
 ('Transatlantic Merry-Go-Round', 0.9676827192306519),
 ('Are We Civilized?', 0.9664505124092102)]